In [139]:
import keras
from keras import layers
import numpy as np
from matplotlib import pyplot

In [140]:
latent_dim = 32
height = 32
width = 32
channels = 3

In [141]:
generator_input = keras.Input(shape=(latent_dim,))

In [142]:
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)

In [143]:
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 32)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_73 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_9 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_74 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_9 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [144]:
discriminator_input = layers.Input(shape=(height, width, channels))

In [145]:
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

In [146]:
discriminator = keras.models.Model(discriminator_input, x)

In [147]:
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_78 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_79 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_80 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [148]:
discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

In [149]:
discriminator.trainable = False

In [150]:
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [151]:
import os
from keras.preprocessing import image

In [152]:
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

In [153]:
x_train = x_train[y_train.flatten() == 6] # this gets only frogs (class 6)
print(x_train.shape)
x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype('float32') / 255.0 # normalize the data

iterations = 10000
batch_size = 20
save_dir = 'D:\\' # where images should be saved

start = 0

(5000, 32, 32, 3)


In [154]:
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,
                                                  latent_dim))
    
    generated_images = generator.predict(random_latent_vectors)
    
    stop = start + batch_size
    real_images = x_train[start: stop]

    combined_images = np.concatenate([generated_images, real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)),
                            np.zeros((batch_size, 1))])
    labels += 0.5 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))
    
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
        
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)
        
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_thing' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_thing' + str(step) + '.png'))

discriminator loss: 0.69245803
adversarial loss: 0.7086368
discriminator loss: 0.57480943
adversarial loss: 2.7242844
discriminator loss: 0.41517362
adversarial loss: 2.3764496
discriminator loss: 0.49742693
adversarial loss: 2.0576534
discriminator loss: 0.56771547
adversarial loss: 1.5338931
discriminator loss: 0.45063597
adversarial loss: 1.4735683
discriminator loss: 0.5920463
adversarial loss: 1.2743175
discriminator loss: 0.5487941
adversarial loss: 1.4234067
discriminator loss: 0.5640694
adversarial loss: 1.5265749
discriminator loss: 0.6057679
adversarial loss: 1.2629954
discriminator loss: 0.55417126
adversarial loss: 1.4020727
discriminator loss: 0.5987967
adversarial loss: 1.3446252
discriminator loss: 0.56235826
adversarial loss: 1.3895983
discriminator loss: 0.5945435
adversarial loss: 1.5748799
discriminator loss: 0.57251436
adversarial loss: 1.4772602
discriminator loss: 0.6059
adversarial loss: 1.281851
discriminator loss: 0.54880476
adversarial loss: 1.3763255
discrimi

OSError: Unable to create file (unable to open file: name = 'gan.h5', errno = 22, error message = 'Invalid argument', flags = 13, o_flags = 302)